# IPv4 Subnetting Calculator

## Importing necessary libraries

In [1]:
import pandas as pd 
import boto3 # boto3 library is AWS SDK (software development kit) for python. It lets python program connect to aws services.

## IPv4 class function

In [2]:
def ip_class(ads_list): # class of IPv4 determined by first octet
    if ads_list[0]>=1 and ads_list[0]<=126: 
        return "Class A" # 1-126 ==> class A
    elif ads_list[0]>=128 and ads_list[0]<=191: 
        return "Class B" # 128-191 ==> class B
    elif ads_list[0]>=192 and ads_list[0]<=223: 
        return "Class C" # 192-223 ==> class C
    elif ads_list[0]>=224 and ads_list[0]<=239: 
        return "Class D" # 224-239 ==> class D
    else: 
        return "Class E" # 240-255 ==> class E

## Class C subnetting function

In [3]:
def class_c(ads_list,choice,quantity): # parameters: ipv4 address (list), network or host needed, quantity of requirement
    if choice=="network": # finding the power of 2 that will cover the network requirements
        start=0
        while quantity > 2**start: 
            start+=1 
        n_bits=start
        h_bits=8-n_bits # in class c total available host bits = 8, but some will be given to network for subnetting.
    else: # finding the power of 2 that will cover the host requirements.
        start = 0
        while quantity + 2 > 2 ** start: # +2 is done because apart from usable host, 2 host will be needed for network id and broadcast id
            start += 1
        h_bits = start
        n_bits = 8 - h_bits # in class c total available host bits = 8, but some will be given to network for subnetting.   
    num_subnets = 2 ** n_bits # total number of subnetts formed
    hosts_per_subnet = (2 ** h_bits) - 2 # total number of usable host per subnet
    print(f"Number of subnets: {num_subnets}")
    print(f"Number of usable host per subnet: {hosts_per_subnet}")
    df=pd.DataFrame(columns=["NID","FUH","LUH","BID"]) # creating a dataframe to store necessary subnet information. 
    # columns in dataframe - NID: network id, FUH: first usable host, LUH: last usable host, BID: broadcast id
    base = ads_list[:3] # N:N:N portion will remain fixed 
    increment = 2 ** h_bits # increment to get to next subnet
    for i in range(num_subnets):
        network_id = f"{base[0]}.{base[1]}.{base[2]}.{i * increment}"
        first_usable = f"{base[0]}.{base[1]}.{base[2]}.{i * increment + 1}" # FUH = NID + 1
        last_usable = f"{base[0]}.{base[1]}.{base[2]}.{(i + 1) * increment - 2}" # LUH = BID - 1 = next NID - 2
        broadcast_id = f"{base[0]}.{base[1]}.{base[2]}.{(i + 1) * increment - 1}" # BID = next NID - 1
        df.loc[len(df)] = [network_id, first_usable, last_usable, broadcast_id] # inserting the subnet in dataframe
    print(f"CIDR: 255.255.255.{256 - increment}/{24+n_bits}") # subnet mask with prefix
    return df # returning the dataframe will all subnets

## Class B subnetting function

In [4]:
def class_b(ads_list,choice,quantity): # parameters: ipv4 address (list), network or host needed, quantity of requirement
    if choice=="network": # finding the power of 2 that will cover the network requirements
        start=0
        while quantity > 2**start: 
            start+=1 
        n_bits=start
        h_bits=16-n_bits # in class b total available host bits = 16, but some will be given to network for subnetting.
    else: # finding the power of 2 that will cover the host requirements.
        start = 0
        while quantity + 2 > 2 ** start: # +2 is done because apart from usable host, 2 host will be needed for network id and broadcast id
            start += 1
        h_bits = start
        n_bits = 16 - h_bits # in class b total available host bits = 16, but some will be given to network for subnetting.   
    num_subnets = 2 ** n_bits # total number of subnetts formed
    hosts_per_subnet = (2 ** h_bits) - 2 # total number of usable host per subnet
    print(f"Number of subnets: {num_subnets}")
    print(f"Number of usable host per subnet: {hosts_per_subnet}")
    df=pd.DataFrame(columns=["NID","FUH","LUH","BID"])  # creating a dataframe to store necessary subnet information. 
    # columns in dataframe - NID: network id, FUH: first usable host, LUH: last usable host, BID: broadcast id 
    base = ads_list[:2] # N:N portion will remain fixed 
    if h_bits>=8: # if host bits >=8, then increments in 3rd octet  
        increment = 2 ** (h_bits % 8) # increment to get to next subnet
        for i in range(num_subnets):
            network_id = f"{base[0]}.{base[1]}.{i*increment}.0" 
            first_usable = f"{base[0]}.{base[1]}.{i*increment}.1" # FUH = NID + 1
            last_usable = f"{base[0]}.{base[1]}.{(i + 1) * increment - 1}.254" # LUH = BID - 1
            broadcast_id = f"{base[0]}.{base[1]}.{(i + 1) * increment - 1}.255"  # BID = next NID - 1
            df.loc[len(df)] = [network_id, first_usable, last_usable, broadcast_id] # inserting the subnet in dataframe
        print(f"CIDR: 255.255.{256 - increment}.0/{16+n_bits}") # subnet mask with prefix  
    else: # if host bits <8, then increments in 4th octet first followed by 3rd octet
        increment=2 ** h_bits # increment to get to next subnet  
        for i in range(256): # outer for loop controls the number of possible values in 3rd octet
            for j in range(256 // increment): # inner for loop controls the number of subnets for a particular value in 3rd octet
                network_id = f"{base[0]}.{base[1]}.{i}.{j * increment}"
                first_usable = f"{base[0]}.{base[1]}.{i}.{j * increment + 1}" # FUH = NID + 1
                last_usable = f"{base[0]}.{base[1]}.{i}.{(j + 1) * increment - 2}" # LUH = BID - 1 = next NID - 2
                broadcast_id = f"{base[0]}.{base[1]}.{i}.{(j + 1) * increment - 1}" # BID = next NID - 1
                df.loc[len(df)] = [network_id, first_usable, last_usable, broadcast_id] # inserting the subnet in dataframe
        print(f"CIDR: 255.255.255.{256 - increment}/{16+n_bits}") # subnet mask with prefix
    return df # returning the dataframe will all subnets

## Class A subnetting function

In [5]:
def class_a(ads_list,choice,quantity): # parameters: ipv4 address (list), network or host needed, quantity of requirement
    if choice=="network": # finding the power of 2 that will cover the network requirements
        start=0
        while quantity > 2**start: 
            start+=1 
        n_bits=start
        h_bits=24-n_bits # in class a total available host bits = 24, but some will be given to network for subnetting.
    else: # finding the power of 2 that will cover the host requirements.
        start = 0
        while quantity + 2 > 2 ** start: # +2 is done because apart from usable host, 2 host will be needed for network id and broadcast id
            start += 1
        h_bits = start
        n_bits = 24 - h_bits # in class a total available host bits = 24, but some will be given to network for subnetting.   
    num_subnets = 2 ** n_bits # total number of subnetts formed
    hosts_per_subnet = (2 ** h_bits) - 2 # total number of usable host per subnet
    print(f"Number of subnets: {num_subnets}")
    print(f"Number of usable host per subnet: {hosts_per_subnet}")
    df=pd.DataFrame(columns=["NID","FUH","LUH","BID"]) # creating a dataframe to store necessary subnet information. 
    # columns in dataframe - NID: network id, FUH: first usable host, LUH: last usable host, BID: broadcast id 
    base=ads_list[:1] # N portion will remain fixed 
    if h_bits >= 16: # if host bits >=16, then increments in 2nd octet  
        increment = 2 ** (h_bits % 8)  # increment to get to next subnet
        for i in range(num_subnets):
            network_id = f"{base[0]}.{i * increment}.0.0"
            first_usable = f"{base[0]}.{i * increment}.0.1" # FUH = NID + 1
            last_usable = f"{base[0]}.{(i + 1) * increment- 1}.255.254" # LUH = BID - 1
            broadcast_id = f"{base[0]}.{(i + 1) * increment - 1}.255.255" # BID = next NID - 1
            df.loc[len(df)] = [network_id, first_usable, last_usable, broadcast_id] # inserting the subnet in dataframe
        print(f"CIDR: 255.{256 - increment}.0.0/{8 + n_bits}") # subnet mask with prefix
    elif h_bits>=8: # if host bits >=8, then increments in 3rd octet first followed by 2nd octet  
        increment=2 ** (h_bits-8) # increment to get to next subnet  
        for i in range(256): # outer for loop controls the number of possible values in 2nd octet
            for j in range(256 // increment):  # inner for loop controls the number of subnets for a particular value in 2nd octet
                network_id = f"{base[0]}.{i}.{j * increment}.0"
                first_usable = f"{base[0]}.{i}.{j * increment}.1" # FUH = NID + 1
                last_usable = f"{base[0]}.{i}.{(j + 1) * increment - 1}.254" # LUH = BID - 1
                broadcast_id = f"{base[0]}.{i}.{(j + 1) * increment - 1}.255" # BID = next NID - 1
                df.loc[len(df)] = [network_id, first_usable, last_usable, broadcast_id] # inserting the subnet in dataframe
        print(f"CIDR: 255.255.{256 - increment}.0/{8 + n_bits}") # subnet mask with prefix
    else: # if host bits <8, then increments in 4th octet first followed by 3rd octet, which is followed by 2nd octet  
        increment = 2 ** h_bits # increment to get to next subnet  
        for k in range(256): # outer for loop controls the number of possible values in 2nd octet
            for i in range(256): # middle for loop controls the number of possible values in 3rd octet
                for j in range(256 // increment):  # inner for loop controls the number of subnets for a particular value in 2nd and 3rd octet
                    network_id = f"{base[0]}.{k}.{i}.{j * increment}"
                    first_usable = f"{base[0]}.{k}.{i}.{j * increment + 1}" # FUH = NID + 1
                    last_usable = f"{base[0]}.{k}.{i}.{(j + 1) * increment - 2}" # LUH = BID - 1 = next NID - 2
                    broadcast_id = f"{base[0]}.{k}.{i}.{(j + 1) * increment - 1}" # BID = next NID - 1
                    df.loc[len(df)] = [network_id, first_usable, last_usable, broadcast_id] # inserting the subnet in dataframe
        print(f"CIDR: 255.255.255.{256 - increment}/{8 + n_bits}") # subnet mask with prefix
    return df # returning the dataframe will all subnets

## Linux server connection function

In [6]:
def linux_server():
    ec2 = boto3.client('ec2') # telling boto3 to look for ec2 instances on cloud
    response = ec2.describe_instances() # information about ec2 instances retrieved and stored
    for reservation in response['Reservations']: # instance id, state, public_ip and private_ip retrieved
        for instance in reservation['Instances']:
            instance_id = instance['InstanceId']
            public_ip = instance.get('PublicIpAddress', 'N/A')
            private_ip = instance.get('PrivateIpAddress', 'N/A')
            state = instance['State']['Name']
        print(f"Instance ID: {instance_id}")
        print(f"  State: {state}")
        print(f"  Public IPv4: {public_ip}") 
        print(f"  Private IPv4: {private_ip}") 
        return (public_ip,private_ip) # returning public and private ip

## Execution script

In [7]:
print("Subnetrix: A Linux Integrated IPv4 Subnet Calculator")
print()
while True:  
    server=input("Choose to fetch IPv4 from linux server or enter manually (server/manual):  ")
    if server=="manual":
        ads=list(map(int,input("Enter IPv4 address (x.x.x.x): ").split("."))) # storing the 4 octets of ip address in a list
    else: 
        server_ip=linux_server()
        server_type=input("Choose public or private IPv4 (public/private): ")
        if server_type=='public': 
            ads=list(map(int,server_ip[0].split("."))) # storing the 4 octets of ip address in a list
        else: 
            ads=list(map(int,server_ip[1].split("."))) # storing the 4 octets of ip address in a list
    if (ads[0]<0 or ads[0]>255) or (ads[1]<0 or ads[1]>255) or (ads[2]<0 or ads[2]>255) or (ads[3]<0 or ads[3]>255): # check for invalid ip address
        print("Invalid IPv4 address") 
        print()
        again=input("Do you want to try another IPv4 (yes/no): ") 
        if again =='yes': 
            continue
        else: 
            break
    if ads[0]==0: # check for reserved addresses (0.x.x.x) representing the network 
        print("This is a reserved IPv4 address signifying the network")
        print()
        again=input("Do you want to try another IPv4 (yes/no): ") 
        if again =='yes': 
            continue
        else: 
            break
    if ads[0]==127: # check for loopback addresses (127.x.x.x) used for testing and diagnosis
        print("This is a loopback IPv4 address")
        print()
        again=input("Do you want to try another IPv4 (yes/no): ") 
        if again =='yes': 
            continue
        else: 
            break
    class_var=ip_class(ads) # determinig the class of ip address
    if class_var=="Class D": # class d ip address for multicast purpose
        print(f"IPv4 class: {class_var}")
        print("This is a multicast IPv4 address")
        print()
        again=input("Do you want to try another IPv4 (yes/no): ") 
        if again =='yes': 
            continue
        else: 
            break
    elif class_var=="Class E": # class e ip address for experimental purpose 
        print(f"Ipv4 class: {class_var}")
        print("This is an experimental IPv4 address")
        print()
        again=input("Do you want to try another IPv4 (yes/no): ") 
        if again =='yes': 
            continue
        else: 
            break
    else:    
        choice=input("Choose host or network (host/network): ")
        if choice == 'host': 
            quantity=int(input(f"Enter the number of usable host needed: "))
        else:    
            quantity=int(input(f"Enter the number of network needed: "))
        if class_var=="Class A": 
            print(f"IPv4 class: {class_var}")
            print("Network-Host Combination: N.H.H.H")
            print("Default Subnet Mask: 255.0.0.0/8")
            df=class_a(ads,choice,quantity)
            df=df.set_index("NID") 
            print(df)
            print()
            again=input("Do you want to try another IPv4 (yes/no): ") 
            if again =='yes': 
                continue
            else: 
                break
        elif class_var=="Class B": 
            print(f"IPv4 class: {class_var}")
            print("Network-Host Combination: N.N.H.H")
            print("Default Subnet Mask: 255.255.0.0/16")
            df=class_b(ads,choice,quantity)
            df=df.set_index("NID")
            print(df)
            print()
            again=input("Do you want to try another IPv4 (yes/no): ") 
            if again =='yes': 
                continue
            else: 
                break
        elif class_var=="Class C": 
            print(f"IPv4 class: {class_var}")
            print("Network-Host Combination: N.N.N.H")
            print("Default Subnet Mask: 255.255.255.0/24")
            df=class_c(ads,choice,quantity)
            df=df.set_index("NID")
            print(df)
            print()
            again=input("Do you want to try another IPv4 (yes/no): ") 
            if again =='yes': 
                continue
            else: 
                break

Subnetrix: A Linux Integrated IPv4 Subnet Calculator



Choose to fetch IPv4 from linux server or enter manually (server/manual):   server


Instance ID: i-08c51cdfbb4e61e3f
  State: running
  Public IPv4: 3.27.213.130
  Private IPv4: 172.31.4.52


Choose public or private IPv4 (public/private):  private
Choose host or network (host/network):  host
Enter the number of usable host needed:  100


IPv4 class: Class B
Network-Host Combination: N.N.H.H
Default Subnet Mask: 255.255.0.0/16
Number of subnets: 512
Number of usable host per subnet: 126
CIDR: 255.255.255.128/25
                           FUH             LUH             BID
NID                                                           
172.31.0.0          172.31.0.1    172.31.0.126    172.31.0.127
172.31.0.128      172.31.0.129    172.31.0.254    172.31.0.255
172.31.1.0          172.31.1.1    172.31.1.126    172.31.1.127
172.31.1.128      172.31.1.129    172.31.1.254    172.31.1.255
172.31.2.0          172.31.2.1    172.31.2.126    172.31.2.127
...                        ...             ...             ...
172.31.253.128  172.31.253.129  172.31.253.254  172.31.253.255
172.31.254.0      172.31.254.1  172.31.254.126  172.31.254.127
172.31.254.128  172.31.254.129  172.31.254.254  172.31.254.255
172.31.255.0      172.31.255.1  172.31.255.126  172.31.255.127
172.31.255.128  172.31.255.129  172.31.255.254  172.31.255.255

[512

Do you want to try another IPv4 (yes/no):  yes
Choose to fetch IPv4 from linux server or enter manually (server/manual):   manual
Enter IPv4 address (x.x.x.x):  192.23.1.5
Choose host or network (host/network):  network
Enter the number of network needed:  4


IPv4 class: Class C
Network-Host Combination: N.N.N.H
Default Subnet Mask: 255.255.255.0/24
Number of subnets: 4
Number of usable host per subnet: 62
CIDR: 255.255.255.192/26
                       FUH           LUH           BID
NID                                                   
192.23.1.0      192.23.1.1   192.23.1.62   192.23.1.63
192.23.1.64    192.23.1.65  192.23.1.126  192.23.1.127
192.23.1.128  192.23.1.129  192.23.1.190  192.23.1.191
192.23.1.192  192.23.1.193  192.23.1.254  192.23.1.255



Do you want to try another IPv4 (yes/no):  no
